In [27]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub
import pandas as pd

# Make sure we don't get any GPU errors
#physical_devices = tf.config.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [28]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [29]:
# Check the shape of the training and test data
print("Train Images Shape:", train_images.shape)
print("Test Images Shape:", test_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Labels Shape:", test_labels.shape)

Train Images Shape: (60000, 28, 28)
Test Images Shape: (10000, 28, 28)
Train Labels Shape: (60000,)
Test Labels Shape: (10000,)


In [30]:
train_images = train_images.reshape(-1,28,28,1).astype("float32") / 255.0
test_images = test_images.reshape(-1,28,28,1).astype("float32") / 255.0

In [31]:
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64,1, padding="valid", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128,1, activation="relu"),
        layers.Conv2D(256,1, activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(10),
    ]
)


model.compile(
        optimizer =tf.keras.optimizers.Adam(),
        loss =keras.losses.SparseCategoricalCrossentropy(from_logits =False),
        metrics =['accuracy']
              )


# Display the model's architecture
model.summary()

model.fit (train_images, train_labels, batch_size=64, epochs=10, verbose =2 )
model.evaluate(test_images, test_labels, batch_size=64, verbose =2 )
model.save('/content')


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 28, 28, 64)        128       
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 14, 14, 128)       8320      
                                                                 
 conv2d_18 (Conv2D)          (None, 14, 14, 256)       33024     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 7, 7, 256)         0         
 ng2D)                                                           
                                                                 
 flatten_5 (Flatten)         (None, 12544)            

In [32]:
model = keras.models.load_model("/content")

In [38]:
# Freeze all model layer weights
model.trainable = False

In [39]:
for layer in model.layers :
  assert layer.trainable == False
  layer.trainable == False

In [40]:
base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output #We didnot read the last layer(dense_6)
final_outputs =layers.Dense(10)(base_outputs)

In [41]:
new_model = keras.Model (inputs = base_inputs, outputs = final_outputs )

In [22]:
#new_model this same with old model . we just want use and work with layers from another model
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 28, 28, 64)        128       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 14, 14, 128)       8320      
                                                                 
 conv2d_15 (Conv2D)          (None, 14, 14, 256)       33024     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 7, 7, 256)         0         
 ng2D)                                                     

In [42]:
new_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [43]:
new_model.fit (train_images, train_labels, batch_size=32, epochs=5, verbose =2 )

Epoch 1/5
1875/1875 - 4s - loss: 0.4461 - accuracy: 0.8682 - 4s/epoch - 2ms/step
Epoch 2/5
1875/1875 - 4s - loss: 0.3372 - accuracy: 0.8946 - 4s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 4s - loss: 0.3224 - accuracy: 0.9001 - 4s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 4s - loss: 0.3142 - accuracy: 0.9021 - 4s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 4s - loss: 0.3109 - accuracy: 0.9030 - 4s/epoch - 2ms/step


In [46]:
#model.evaluate(test_images, test_labels, batch_size=64, verbose =2 )

In [47]:
# =================================================== #
#                Pretrained Keras Model               #
# =================================================== #

In [50]:
# Random data for demonstration (3 examples w. 3 classes)
x = tf.random.normal (shape =(3, 299, 299, 3))
y = tf.constant([0, 1, 2])

In [51]:
model = keras.applications.InceptionV3(include_top=True)
#model.summary()

96112376/96112376 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 149, 149, 32)         864       ['input_9[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d_19[0][0]']           
 Normalization)                                                                                   
                                                                                                  
 activation (Activatio

In [52]:
# for input you can also do model.input,
# then for base_outputs you can obviously
# choose other than simply removing the last one :)
base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
classifier = layers.Dense(3)(base_outputs)
new_model = keras.Model(inputs=base_inputs, outputs=classifier)

In [54]:
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [55]:
new_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 149, 149, 32)         864       ['input_9[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d_19[0][0]']           
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0]

In [56]:
new_model.fit(x, y, epochs=15, verbose=2)

Epoch 1/15
1/1 - 44s - loss: 1.0876 - accuracy: 0.0000e+00 - 44s/epoch - 44s/step
Epoch 2/15
1/1 - 0s - loss: 0.0409 - accuracy: 1.0000 - 67ms/epoch - 67ms/step
Epoch 3/15
1/1 - 0s - loss: 1.4950e-04 - accuracy: 1.0000 - 67ms/epoch - 67ms/step
Epoch 4/15
1/1 - 0s - loss: 4.3867e-05 - accuracy: 1.0000 - 76ms/epoch - 76ms/step
Epoch 5/15
1/1 - 0s - loss: 2.3483e-05 - accuracy: 1.0000 - 67ms/epoch - 67ms/step
Epoch 6/15
1/1 - 0s - loss: 1.4980e-05 - accuracy: 1.0000 - 66ms/epoch - 66ms/step
Epoch 7/15
1/1 - 0s - loss: 1.0411e-05 - accuracy: 1.0000 - 66ms/epoch - 66ms/step
Epoch 8/15
1/1 - 0s - loss: 8.0267e-06 - accuracy: 1.0000 - 69ms/epoch - 69ms/step
Epoch 9/15
1/1 - 0s - loss: 6.3975e-06 - accuracy: 1.0000 - 67ms/epoch - 67ms/step
Epoch 10/15
1/1 - 0s - loss: 5.2452e-06 - accuracy: 1.0000 - 67ms/epoch - 67ms/step
Epoch 11/15
1/1 - 0s - loss: 4.3710e-06 - accuracy: 1.0000 - 68ms/epoch - 68ms/step
Epoch 12/15
1/1 - 0s - loss: 3.7352e-06 - accuracy: 1.0000 - 66ms/epoch - 66ms/step
Epoch 

In [57]:
# ================================================= #
#                Pretrained Hub Model               #
# ================================================= #

In [58]:
# Random data for demonstration (3 examples w. 3 classes)
x = tf.random.normal(shape=(3, 299, 299, 3))
y = tf.constant([0, 1, 2])

In [59]:
url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4"

In [67]:
base_model = hub.KerasLayer(url, input_shape = (299, 299, 3))
base_model.trainable = False

In [68]:
model = keras.Sequential(
    [
        base_model,
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

In [69]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 2048)              21802784  
                                                                 
 dense_18 (Dense)            (None, 128)               262272    
                                                                 
 dense_19 (Dense)            (None, 64)                8256      
                                                                 
 dense_20 (Dense)            (None, 10)                650       
                                                                 
Total params: 22073962 (84.21 MB)
Trainable params: 271178 (1.03 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________


In [70]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [71]:
model.fit(x, y, batch_size=32, epochs=15, verbose=2)

Epoch 1/15
1/1 - 6s - loss: 2.7353 - accuracy: 0.0000e+00 - 6s/epoch - 6s/step
Epoch 2/15
1/1 - 0s - loss: 2.1116 - accuracy: 0.3333 - 42ms/epoch - 42ms/step
Epoch 3/15
1/1 - 0s - loss: 1.8080 - accuracy: 0.6667 - 42ms/epoch - 42ms/step
Epoch 4/15
1/1 - 0s - loss: 1.6033 - accuracy: 0.6667 - 41ms/epoch - 41ms/step
Epoch 5/15
1/1 - 0s - loss: 1.4374 - accuracy: 0.6667 - 40ms/epoch - 40ms/step
Epoch 6/15
1/1 - 0s - loss: 1.2906 - accuracy: 0.6667 - 40ms/epoch - 40ms/step
Epoch 7/15
1/1 - 0s - loss: 1.1364 - accuracy: 0.6667 - 39ms/epoch - 39ms/step
Epoch 8/15
1/1 - 0s - loss: 0.9895 - accuracy: 0.6667 - 31ms/epoch - 31ms/step
Epoch 9/15
1/1 - 0s - loss: 0.8521 - accuracy: 0.6667 - 30ms/epoch - 30ms/step
Epoch 10/15
1/1 - 0s - loss: 0.7306 - accuracy: 1.0000 - 36ms/epoch - 36ms/step
Epoch 11/15
1/1 - 0s - loss: 0.6270 - accuracy: 1.0000 - 30ms/epoch - 30ms/step
Epoch 12/15
1/1 - 0s - loss: 0.5454 - accuracy: 1.0000 - 29ms/epoch - 29ms/step
Epoch 13/15
1/1 - 0s - loss: 0.4756 - accuracy: 1